### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

### Preliminary Data 

In [3]:
df = pd.read_csv('./DATA/forestcleaned.csv')

In [4]:
df.head()

,Unnamed: 0,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month,day
0,0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,3,5
1,1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10,2
2,2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10,6
3,3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,3,5
4,4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,3,7


In [5]:
df = df.drop('Unnamed: 0', axis=1)

In [6]:
df.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month,day
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,3,5
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10,2
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10,6
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,3,5
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,3,7


#### Split into Features and Target

In [7]:
X = df.drop('area', axis=1)

In [8]:
y = df['area']

In [9]:
X.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,month,day
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,3,5
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,10,2
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,10,6
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,3,5
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,3,7


### Train-Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### Linear Regression

In [11]:
linear_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('linear', LinearRegression())
])

In [12]:
param_grid_lin = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],
    'linear__fit_intercept': [True, False],
    'linear__positive': [True, False]
}

In [13]:
grid_search_lin = GridSearchCV(linear_pipeline, param_grid_lin, cv=5, scoring='r2')

In [14]:
grid_search_lin.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('linear', LinearRegression())]),
             param_grid={'linear__fit_intercept': [True, False],
                         'linear__positive': [True, False],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler(), None]},
             scoring='r2')

In [15]:
best_model_lin = grid_search_lin.best_estimator_

In [16]:
y_lin_preds = best_model_lin.predict(X_test)

### Polynomial Regression

In [17]:
poly_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('poly', PolynomialFeatures()), 
    ('linear', LinearRegression())
])

In [18]:
param_grid_poly = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()], 
    'poly__degree': [2,3,4], 
    'poly__interaction_only': [False, True],
    'linear__fit_intercept': [True, False], 
    'linear__positive': [True, False]
}

In [19]:
grid_poly = GridSearchCV(poly_pipeline, param_grid_poly, cv=5, scoring='r2')

In [20]:
grid_poly.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
6 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 473, in fit
    self._final_estimator.fit(

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('linear', LinearRegression())]),
             param_grid={'linear__fit_intercept': [True, False],
                         'linear__positive': [True, False],
                         'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler()]},
             scoring='r2')

In [21]:
best_model_poly = grid_poly.best_estimator_

In [22]:
y_pred_poly = best_model_poly.predict(X_test)

### Ridge Regression

In [24]:
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

In [25]:
param_grid_ridge = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],
    'poly__degree': [2,3,4],
    'poly__interaction_only': [False, True], 
    'ridge__alpha': [0.1, 1.0, 10, 100],
    'ridge__fit_intercept': [True, False]
}

In [26]:
ridge_grid = GridSearchCV(ridge_pipe, param_grid_ridge, cv=5, scoring='r2')

In [27]:
ridge_grid.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_mo

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.90746e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.44608e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.39384e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=6.13183e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:21

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_mo

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=1.97771e-26): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\admin

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('ridge', Ridge())]),
             param_grid={'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'ridge__alpha': [0.1, 1.0, 10, 100],
                         'ridge__fit_intercept': [True, False],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler(), None]},
             scoring='r2')

In [28]:
best_model_ridge = ridge_grid.best_estimator_

In [29]:
y_pred_ridge = best_model_ridge.predict(X_test)

### Lasso Regression

In [31]:
lasso_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('lasso', Lasso())
])

In [32]:
param_lasso = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],
    'poly__degree': [2,3,4], 
    'poly__interaction_only': [False, True],
    'lasso__alpha': [0.001,0.01,0.1,1,10],
    'lasso__fit_intercept': [True,False],
}

In [33]:
grid_lasso = GridSearchCV(lasso_pipe, param_lasso, cv=5, scoring='r2')

In [34]:
grid_lasso.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.254e+05, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.939e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.764e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.113e+05, tolerance: 8.545e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.352e+04, tolerance: 3.183e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.088e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.000e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.245e+04, tolerance: 8.545e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.484e+04, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.865e+05, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.704e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.224e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.226e+05, tolerance: 8.662e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.767e+04, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.912e+04, toleranc

...

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.498e+05, tolerance: 9.021e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.147e+05, tolerance: 3.535e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.973e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.380e+05, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.215e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.344e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.422e+05, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.219e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.595e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.595e+05, tolerance: 9.010e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.158e+04, tolerance: 8.262e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.794e+05, toleranc

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('lasso', Lasso())]),
             param_grid={'lasso__alpha': [0.001, 0.01, 0.1, 1, 10],
                         'lasso__fit_intercept': [True, False],
                         'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler(), None]},
             scoring='r2')

In [35]:
best_model_lasso = grid_lasso.best_estimator_

In [36]:
y_pred_lasso = best_model_lasso.predict(X_test)

### Elastic Net Regression

In [37]:
from sklearn.linear_model import ElasticNet

In [38]:
elastic_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('elasticnet', ElasticNet())
])

In [40]:
elastic_param = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
    'poly__degree': [2,3,4],
    'poly__interaction_only': [False, True],
    'elasticnet__alpha': [0.001,0.01,0.1,1,10],
    'elasticnet__l1_ratio': [0.2,0.5,0.8],
    'elasticnet__fit_intercept': [True, False]
}

In [41]:
elastic_grid = GridSearchCV(elastic_pipe, elastic_param, cv=5, scoring='r2')

In [42]:
elastic_grid.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.451e+05, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.108e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.163e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.826e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.188e+04, tolerance: 8.545e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.268e+04, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.934e+05, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.198e+05, tolerance: 8.545e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.212e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.743e+04, tolerance: 7.797e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.696e+04, tolerance: 8.545e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.120e+04, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.405e+05, tolerance: 8.662e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.264e+05, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.279e+05, toleranc

...

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.339e+05, tolerance: 9.021e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.982e+02, tolerance: 3.535e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.909e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.576e+03, tolerance: 9.230e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.532e+05, tolerance: 9.010e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.011e+05, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.613e+03, tolerance: 8.662e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.270e+04, tolerance: 8.498e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.315e+04, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.960e+04, tolerance: 9.010e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.733e+03, tolerance: 8.262e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.668e+02, toleranc

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('elasticnet', ElasticNet())]),
             param_grid={'elasticnet__alpha': [0.001, 0.01, 0.1, 1, 10],
                         'elasticnet__fit_intercept': [True, False],
                         'elasticnet__l1_ratio': [0.2, 0.5, 0.8],
                         'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler()]},
             scoring='r2')

In [43]:
best_model_elastic = elastic_grid.best_estimator_

In [45]:
y_pred_elastic = best_model_elastic.predict(X_test)

## Metrics

In [64]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score, max_error
import math

In [79]:
def adjusted_r2_score(y_test, y_pred, X_test):
    r2 = r2_score(y_test, y_pred)  
    n = len(y_test)  
    p = X_test.shape[1]  
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)  
    return adjusted_r2

In [63]:
def mean_absolute_percentage_error(y_test, y_pred):
    return np.mean(np.abs((y_test-y_pred)/y_test))*100

### Linear Model

In [74]:
mae_lin = mean_absolute_error(y_test, y_lin_preds)
mse_lin = mean_squared_error(y_test, y_lin_preds)
rmse_lin = math.sqrt(mse_lin)
r2_lin = r2_score(y_test, y_lin_preds)
adjustr2_lin = adjusted_r2_score(y_test, y_lin_preds, X_test)
mape_lin = mean_absolute_percentage_error(y_test, y_lin_preds)
evs_lin = explained_variance_score(y_test, y_lin_preds)
maxe_lin = max_error(y_test, y_lin_preds)

### Polynomial Regression

In [85]:
mae_pol = mean_absolute_error(y_test, y_pred_poly)
mse_pol = mean_squared_error(y_test, y_pred_poly)
rmse_pol = math.sqrt(mse_pol)
r2_pol = r2_score(y_test, y_pred_poly)
adjustr2_pol = adjusted_r2_score(y_test, y_pred_poly, X_test)
mape_pol = mean_absolute_percentage_error(y_test, y_pred_poly)
evs_pol = explained_variance_score(y_test, y_pred_poly)
maxe_pol = max_error(y_test, y_pred_poly)

### Ridge Regression

In [81]:
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = math.sqrt(mse_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
adjustr2_ridge = adjusted_r2_score(y_test, y_pred_ridge, X_test)
mape_ridge = mean_absolute_percentage_error(y_test, y_pred_ridge)
evs_ridge = explained_variance_score(y_test, y_pred_ridge)
maxe_ridge = max_error(y_test, y_pred_ridge)

### Lasso Regression

In [82]:
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = math.sqrt(mse_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
adjustedr2_lasso = adjusted_r2_score(y_test, y_pred_lasso, X_test)
mape_lasso = mean_absolute_percentage_error(y_test, y_pred_lasso)
evs_lasso = explained_variance_score(y_test, y_pred_lasso)
maxe_lasso = max_error(y_test, y_pred_ridge)

### Elastic Net Regression 

In [80]:
mae_elastic = mean_absolute_error(y_test, y_pred_elastic)
mse_elastic = mean_squared_error(y_test, y_pred_elastic)
rmse_elastic = math.sqrt(mse_elastic)
r2_elastic = r2_score(y_test, y_pred_elastic)
adjustedr2_elastic = adjusted_r2_score(y_test, y_pred_elastic, X_test)
mape_elastic = mean_absolute_percentage_error(y_test, y_pred_elastic)
evs_elastic = explained_variance_score(y_test, y_pred_elastic)
maxe_elastic = max_error(y_test, y_pred_elastic)

## Alignment of All Metrics

In [86]:
metrics_dict = {
    'Linear': [
        mae_lin, mse_lin, rmse_lin, r2_lin,
        adjustr2_lin, mape_lin, evs_lin, maxe_lin],
    'Polynomial': [
        mae_pol, mse_lin, rmse_pol, r2_pol,
        adjustr2_pol, mape_pol, evs_pol, maxe_pol
    ],
    'Ridge': [
        mae_ridge, mse_ridge, rmse_ridge, r2_ridge, 
        adjustr2_ridge, mape_ridge, evs_ridge, maxe_ridge
    ],
    'Lasso': [
        mae_lasso, mse_lasso, rmse_lasso, r2_lasso, 
        adjustedr2_lasso, mape_lasso, evs_lasso, maxe_lasso
    ],
    'ElasticNet': [
        mae_elastic, mse_elastic, rmse_elastic, r2_elastic, 
        adjustedr2_elastic, mape_elastic, evs_elastic, maxe_elastic
    ]
}

In [87]:
metrics_df = pd.DataFrame(metrics_dict, 
                         index=['MAE', 'MSE', 'RMSE',
                               'R2', 'Adjusted R2',
                               'MAPE', 'EVS', 'MAXError'])

In [88]:
metrics_df

,Linear,Polynomial,Ridge,Lasso,ElasticNet
MAE,3.269952e+01,3.318532e+01,3.226107e+01,3.171971e+01,3.169401e+01
MSE,2.249377e+04,2.249377e+04,2.240863e+04,2.254033e+04,2.253907e+04
RMSE,1.499792e+02,1.495578e+02,1.496951e+02,1.501344e+02,1.501302e+02
R2,-6.852423e-03,-1.202230e-03,-3.041758e-03,-8.936733e-03,-8.880405e-03
Adjusted R2,-3.166532e-01,-3.092645e-01,-3.116700e-01,-3.193788e-01,-3.193051e-01
MAPE,inf,inf,inf,inf,inf
EVS,2.877534e-03,7.181748e-03,7.012266e-03,2.808852e-03,2.781093e-03
MAXError,1.076294e+03,1.072540e+03,1.074230e+03,1.074230e+03,1.077918e+03
